<a href="https://colab.research.google.com/github/GabrielYamil/TP1_colab/blob/main/TP1/Ejercicio_3_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

gestión de eventos y probabilidades

In [ ]:
%%writefile EventoParanormal.java
import java.util.Random;

public class EventoParanormal {
    private static final String[] eventos = {
        "Sin actividad",
        "Movimiento detectado",
        "Anomalía térmica",
        "Sombra extraña",
        "Ruido detectado"
    };

    private static final double[] probabilidades = {
        0.5,  // Sin actividad (50%)
        0.7,  // Movimiento detectado (20%)
        0.8,  // Anomalía térmica (10%)
        0.9,  // Sombra extraña (10%)
        1.0   // Ruido detectado (10%)
    };

    private static final Random random = new Random();

    public static String generarEvento() {
        double r = random.nextDouble();
        for (int i = 0; i < probabilidades.length; i++) {
            if (r < probabilidades[i]) {
                return eventos[i];
            }
        }
        return eventos[0];
    }

    public static boolean esParanormal(String evento) {
        return !evento.equals("Sin actividad");
    }
}

Writing EventoParanormal.java


log con timestam

In [ ]:
%%writefile LoggerCamara.java
import java.io.FileWriter;
import java.io.IOException;
import java.io.PrintWriter;
import java.time.LocalDateTime;
import java.time.format.DateTimeFormatter;

public class LoggerCamara {
    private final PrintWriter writer;
    private final DateTimeFormatter dtf = DateTimeFormatter.ofPattern("yyyy-MM-dd HH:mm:ss");

    public LoggerCamara(String fileName) throws IOException {
        this.writer = new PrintWriter(new FileWriter(fileName));
    }

    public void log(String mensaje) {
        String timestamp = LocalDateTime.now().format(dtf);
        writer.println(timestamp + " | " + mensaje);
        writer.flush();
    }

    public void cerrar() {
        writer.close();
    }
}

Writing LoggerCamara.java


In [ ]:
%%writefile CamaraProceso.java
public class CamaraProceso {
    public static void main(String[] args) {
        if (args.length < 4) {
            System.out.println("Uso: java CamaraProceso <id> <zona> <duracion> <frecuencia>");
            System.exit(1);
        }

        int id = Integer.parseInt(args[0]);
        String zona = args[1];
        int duracion = Integer.parseInt(args[2]);
        int frecuencia = Integer.parseInt(args[3]);

        if (duracion <= 0 || frecuencia <= 0) {
            System.err.println("Duración y frecuencia deben ser mayores que cero.");
            System.exit(1);
        }

        int eventosParanormales = 0;
        String logFile = "camara_" + id + "_" + zona + ".log";

        try {
            LoggerCamara logger = new LoggerCamara(logFile);
            long inicio = System.currentTimeMillis();
            long tiempoTotal = duracion * 1000L;

            while ((System.currentTimeMillis() - inicio) < tiempoTotal) {
                String evento = EventoParanormal.generarEvento();
                logger.log("CÁMARA " + id + " | ZONA: " + zona + " | EVENTO: " + evento);

                if (EventoParanormal.esParanormal(evento)) {
                    eventosParanormales++;
                }

                try {
                    Thread.sleep(frecuencia * 1000L);
                } catch (InterruptedException e) {
                    logger.log("Cámara " + id + " interrumpida.");
                    break;
                }
            }

            logger.log("CÁMARA " + id + " finaliza con " + eventosParanormales + " eventos paranormales detectados.");
            logger.cerrar();
        } catch (Exception e) {
            System.err.println("Error en cámara " + id + ": " + e.getMessage());
            e.printStackTrace();
        }
    }
}

Writing CamaraProceso.java


In [ ]:
%%writefile MansionDercetoMonitor.java
import java.io.*;
import java.util.*;

public class MansionDercetoMonitor {
    public static void main(String[] args) {
        if (args.length < 2) {
            System.out.println("Uso: java MansionDercetoMonitor <duracion> <frecuencia>");
            System.exit(1);
        }

        int duracion = 0, frecuencia = 0;

        try {
            duracion = Integer.parseInt(args[0]);
            frecuencia = Integer.parseInt(args[1]);

            if (duracion <= 0 || frecuencia <= 0) {
                System.err.println("Duración y frecuencia deben ser mayores que cero.");
                System.exit(1);
            }
        } catch (NumberFormatException e) {
            System.err.println("La duración y frecuencia deben ser enteros.");
            System.exit(1);
        }

         String[] zonas = {
            "Sotano",
            "Atico",
            "Cocina",
            "Dormitorio",
            "Jardin",
            "Mausoleo"
        };

        List<Process> procesos = new ArrayList<>();
        List<String> logFiles = new ArrayList<>();

        for (int i = 0; i < zonas.length; i++) {
            String zona = zonas[i];
            String logFile = "camara_" + (i + 1) + "_" + zona + ".log";
            logFiles.add(logFile);

            try {
                ProcessBuilder pb = new ProcessBuilder(
                        "java", "CamaraProceso",
                        String.valueOf(i + 1),
                        zona,
                        String.valueOf(duracion),
                        String.valueOf(frecuencia)
                );
                pb.directory(new File("."));
                Process proceso = pb.start();
                procesos.add(proceso);
            } catch (IOException e) {
                System.err.println("Error iniciando cámara " + (i + 1));
                e.printStackTrace();
            }
        }

        // Esperar a que terminen todas las cámaras
        for (Process proceso : procesos) {
            try {
                proceso.waitFor();
            } catch (InterruptedException e) {
                System.err.println("Error esperando una cámara.");
                e.printStackTrace();
            }
        }

         System.out.println("\nMonitoreo finalizado. Lectura de logs:");

        System.out.println("\n======= LOG DE TODAS LAS CÁMARAS =======\n");
        for (String file : logFiles) {
            System.out.println(">>> " + file);
            try (BufferedReader reader = new BufferedReader(new FileReader(file))) {
                String linea;
                while ((linea = reader.readLine()) != null) {
                    System.out.println(linea);
                }
            } catch (IOException e) {
                System.err.println("No se pudo leer el archivo: " + file);
            }
            System.out.println(); // Espacio entre logs
        }

        System.out.println("Monitoreo completo.");
    }
}

Writing MansionDercetoMonitor.java


In [ ]:
!ls -la

total 32
drwxr-xr-x 1 root root 4096 May  1 19:19 .
drwxr-xr-x 1 root root 4096 May  1 19:17 ..
-rw-r--r-- 1 root root 1794 May  1 19:18 CamaraProceso.java
drwxr-xr-x 4 root root 4096 Apr 30 13:37 .config
-rw-r--r-- 1 root root  969 May  1 19:18 EventoParanormal.java
-rw-r--r-- 1 root root  695 May  1 19:18 LoggerCamara.java
-rw-r--r-- 1 root root 2948 May  1 19:19 MansionDercetoMonitor.java
drwxr-xr-x 1 root root 4096 Apr 30 13:37 sample_data


In [ ]:
!javac *.java

In [ ]:
!java MansionDercetoMonitor 10 2


Monitoreo finalizado. Lectura de logs:

======= LOG DE TODAS LAS CÁMARAS =======

>>> camara_1_Sotano.log
2025-05-01 19:19:30 | CÁMARA 1 | ZONA: Sotano | EVENTO: Sin actividad
2025-05-01 19:19:32 | CÁMARA 1 | ZONA: Sotano | EVENTO: Sombra extraña
2025-05-01 19:19:34 | CÁMARA 1 | ZONA: Sotano | EVENTO: Movimiento detectado
2025-05-01 19:19:36 | CÁMARA 1 | ZONA: Sotano | EVENTO: Sin actividad
2025-05-01 19:19:38 | CÁMARA 1 | ZONA: Sotano | EVENTO: Sin actividad
2025-05-01 19:19:40 | CÁMARA 1 finaliza con 2 eventos paranormales detectados.

>>> camara_2_Atico.log
2025-05-01 19:19:30 | CÁMARA 2 | ZONA: Atico | EVENTO: Anomalía térmica
2025-05-01 19:19:32 | CÁMARA 2 | ZONA: Atico | EVENTO: Anomalía térmica
2025-05-01 19:19:34 | CÁMARA 2 | ZONA: Atico | EVENTO: Movimiento detectado
2025-05-01 19:19:36 | CÁMARA 2 | ZONA: Atico | EVENTO: Sin actividad
2025-05-01 19:19:38 | CÁMARA 2 | ZONA: Atico | EVENTO: Ruido detectado
2025-05-01 19:19:40 | CÁMARA 2 finaliza con 4 eventos paranormales detec